In [1]:
import os 
import sys 
import numpy as np 
import pandas as pd 
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import tensorflow as tf

2022-10-04 17:33:09.397660: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-10-04 17:33:11.814665: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-10-04 17:33:12.846245: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-10-04 17:33:35.480668: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; 

In [2]:
with open('train.tsv', 'r') as f: 
    data = f.readlines()

In [3]:
for i in range(len(data)): 
    data[i] = data[i].split('\t')
    data[i][0] = data[i][0].lower()
    data[i][1] = data[i][1].strip('\n').lower().split('-')

In [4]:
label_pool = []
char_pool = [] 
for i in range(len(data)):
    label_pool+= data[i][1]
    char_pool += list(data[i][0].lower())
label_pool = list(set(label_pool))
char_pool = list(set(char_pool))




In [5]:
print(len(label_pool))
print(len(char_pool))

32
27


In [6]:
set(label_pool).difference(set(char_pool))

{'an', 'ch', 'en', 'ng', 'on', 'ou', 'oun', 'ui'}

In [7]:
def create_labels(data_list):
    single_tag = 0 
    double_tag = 0 
    triple_tag = 0
    label_list = [] 
    for i in range(len(data_list)):
        temp_labels = []
        for j in range(len(data_list[i][1])): 
            if len(data_list[i][1][j])==1:
                temp_labels += ['B']
                single_tag += 1 
            elif len(data_list[i][1][j])==2:
                temp_labels += ['B','I']
                double_tag += 1 
            elif len(data_list[i][1][j])==3:
                temp_labels += ['B','I','I']
                triple_tag += 1 
        label_list.append(temp_labels)
    print("The number of single tags are: ", single_tag)
    print("The number of double tags are: ", double_tag)
    print("The number of triple tags are: ", triple_tag)
    return label_list

In [15]:
labels = create_labels(data)

The number of single tags are:  72096
The number of double tags are:  7592
The number of triple tags are:  8


In [18]:
encoded_label_pool = []
for e in labels:
    encoded_label_pool+= e

encoded_label_pool = list(set(encoded_label_pool))


In [11]:
def pad(char_list):
    max_len = 0 
    for i in range(len(char_list)):
        if len(char_list[i])>max_len: 
            max_len = len(char_list[i])
    for i in range(len(char_list)): 
        if len(char_list[i])< max_len: 
            for _ in range(max_len- len(char_list[i])): 
                char_list[i] += ['<PAD>']
    return char_list

In [22]:
def encode_words(char_list, char_dict):
    encoded_chars = [] 
    for i in range(len(char_list)):
        temp_encoded = [] 
        for j in range(len(char_list[i])):
            temp_encoded.append(char_dict[char_list[i][j]]) 
        encoded_chars.append(np.array(temp_encoded))   # .reshape(-1,1)
    return encoded_chars

In [19]:
label_dict = {}
label_dict['<PAD>'] = 0
counter = 1 
for e in encoded_label_pool: 
    label_dict[e] = counter 
    counter += 1 
    
invers_label_dict = {v:k for k,v in label_dict.items()}

In [20]:
invers_label_dict

{0: '<PAD>', 1: 'I', 2: 'B'}

In [21]:
padded_labels = pad(labels)

In [23]:
encoded_labels  = encode_words(padded_labels, label_dict)

In [24]:
encoded_labels[0]

array([2, 2, 1, 2, 2, 2, 2, 2, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [25]:
char_dict = {}
char_dict['<PAD>'] = 0
char_dict['<UNK>'] = 1 
counter = 2 
for e in char_pool: 
    char_dict[e] = counter 
    counter += 1 

In [26]:
invers_label_dictchar_list = [] 
for e in data: 
    char_list.append(list(e[0]))

In [27]:
char_list[0]

['k', 'o', 'n', 's', 'i', 'l', 't', 'a', 'n']

In [28]:
paded_list = pad(char_list)

In [29]:
encoded_data = encode_words(paded_list, char_dict)

In [30]:
encoded_data[0:3]

[array([15,  9, 27, 17, 26,  4, 19, 21, 27,  0,  0,  0,  0,  0,  0,  0,  0,
         0]),
 array([28, 12, 20,  9, 19, 18, 21,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0]),
 array([17,  9, 17, 14,  9, 20, 18,  9,  2, 12, 17, 14,  9, 27, 16,  4,  0,
         0])]

In [31]:
X_train, X_test, y_train, y_test = train_test_split(np.array(encoded_data), np.array(encoded_labels), test_size=0.2, random_state=42)

In [108]:
class Masked_Accuracy(tf.keras.metrics.SparseCategoricalAccuracy):
    def __init__(self,):
        super(Masked_Accuracy, self).__init__()

    def call(self, y_true, y_pred):
        total = 0.0
        for i in range(len(y_true)):
            y_t = y_true[i][y_true[i]!=0]
            y_p = tf.argmax(y_pred[i][:len(y_t)], axis = -1)
            self.update_state(y_t, y_p)
            total += m.result()
        return total/tf.cast(len(y_true), tf.float32)

In [99]:
def masked_accuracy(y_true, y_pred):
    total = 0.0
    for i in range(len(y_true)):
        y_t = y_true[i][y_true[i]!=0]
        y_p = tf.argmax(y_pred[i], axis = -1)[:len(y_t)]
        m = tf.keras.metrics.Accuracy()
        m.update_state(y_t, y_p)
        total += m.result()
    return total/tf.cast(len(y_true), tf.float32)
    

In [111]:
input_chars_input = len(char_dict)
embedding_vector_length = 16

inputs = tf.keras.Input(shape=(X_train.shape[-1]), name="word_input")
x = tf.keras.layers.Embedding(input_chars_input, embedding_vector_length, input_length = 1)(inputs)
print(x.shape)

# x = tf.keras.layers.LSTM(128,return_sequences=True)(x) # 128
# x = tf.keras.layers.Dropout(0.2)(x)

x = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128,return_sequences=True))(x) # 128
print(x.shape)
x = tf.keras.layers.Dropout(0.2)(x)

# dense_layer = tf.keras.layers.Dense(64, activation="tanh", name="dense_1")
# x = tf.keras.layers.TimeDistributed(dense_layer)(x)

output_layer = tf.keras.layers.Dense(3, activation="softmax", name="predictions")
outputs = tf.keras.layers.TimeDistributed(output_layer)(x)

model = tf.keras.Model(inputs=inputs, outputs=outputs)


# recall = tf.keras.metrics.Recall(class_id=4)
scce = tf.keras.metrics.SparseCategoricalCrossentropy()

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=[Masked_Accuracy(),scce]) # recall, sparse_categorical_cross_entropy
print(model.summary())

model.fit(X_train, y_train, validation_split =0.2, epochs=10, batch_size=32)

scores = model.evaluate(X_test, y_test, verbose=0)
print("The final accuracy in test set is: %.2f%%" % (scores[1]*100))


(None, 18, 16)
(None, 18, 256)
Model: "model_16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 word_input (InputLayer)     [(None, 18)]              0         
                                                                 
 embedding_16 (Embedding)    (None, 18, 16)            464       
                                                                 
 bidirectional_16 (Bidirecti  (None, 18, 256)          148480    
 onal)                                                           
                                                                 
 dropout_16 (Dropout)        (None, 18, 256)           0         
                                                                 
 time_distributed_16 (TimeDi  (None, 18, 3)            771       
 stributed)                                                      
                                                                 
Total params: 149,715
Train

In [80]:
scores = model.evaluate(X_test, y_test, verbose=0)
print("The final accuracy in test set is: %.2f%%" % (scores[1]*100))


[array([0, 0, 1, 0, 0, 0, 0, 0, 1]),
 array([0, 0, 0, 0, 0, 0, 0]),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 array([0, 0, 0, 0, 0, 0, 0]),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0])]

In [164]:
def decode(y_true, y_pred, inverse_dictionary, sparse = True):
    decoded_list = [] 
    for i in range(len(y_pred)): 
        temp_len = len(y_true[i][y_true[i]!=0])
        temp_prediction = y_pred[i]
        if sparse: 
            temp_prediction = tf.argmax(temp_prediction, axis = -1).numpy()
        temp_prediction = temp_prediction[:temp_len]
        temp_str = '' 
        for j in range(len(temp_prediction)):
            temp_str += inverse_dictionary[temp_prediction[j]] + '-'
        decoded_list.append(temp_str.strip('-'))
    return decoded_list

In [122]:
predicted_test = model.predict(X_test)

81/81 [==============================] - 0s 4ms/step


In [165]:
decoded_predictions = decode(y_test, predicted_test, invers_label_dict)

In [166]:
decoded_labels = decode(y_test, y_test, invers_label_dict, sparse = False )

In [167]:
print("The predictions for the first sample is as follows: ", decoded_predictions[0])
print("The labels for the first sample is as follows: ", decoded_labels[0])

The predictions for the first sample is as follows:  B-B-B-B-I-B-B-B-B
The labels for the first sample is as follows:  B-B-B-B-I-B-B-B-B


In [263]:
def precision(y_true, y_pred, tp_label , sparse = True):
    decoded_list = [] 
    tp = 0 
    all_pred_p = 0 
    for i in range(len(y_pred)): 
        temp_len = len(y_true[i][y_true[i]!=0])
        temp_prediction = y_pred[i]
        if sparse: 
            temp_prediction = tf.argmax(temp_prediction, axis = -1).numpy()
        temp_prediction = temp_prediction[:temp_len]
        label = y_test[i][:temp_len]
        tp += np.sum(np.multiply(label==tp_label, temp_prediction==tp_label))
        all_pred_p += np.where(temp_prediction==tp_label)[0].shape[0]
        
    return tp/(all_pred_p + np.finfo(float).eps)


def recall(y_true, y_pred, tp_label , sparse = True):
    decoded_list = [] 
    tp = 0
    all_p = 0 
    for i in range(len(y_pred)): 
        temp_len = len(y_true[i][y_true[i]!=0])
        temp_prediction = y_pred[i]
        if sparse: 
            temp_prediction = tf.argmax(temp_prediction, axis = -1).numpy()
        temp_prediction = temp_prediction[:temp_len]
        label = y_test[i][:temp_len]
        tp += np.sum(np.multiply(label==tp_label, temp_prediction==tp_label))
        all_p += label[label==tp_label].shape[0]
        
    return tp/(all_p + np.finfo(float).eps)


def f1(y_true, y_pred, tp_label):
    '''
    y_true: showing the true labels 
    y_pred: showing the predicted values
    tp_label: showing the p
        p: precison 
        r: recall 
        
    '''
    p = precision(y_true, y_pred, tp_label)
    r = recall(y_true, y_pred, tp_label)
    return 2 * ((p * r) / (p + r +  np.finfo(float).eps))

In [267]:
f1(y_test, predicted_test, 1)

0.9894378194207836